In [1]:
import pandas as pd
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from collections import Counter
from glob import glob
from tqdm import tqdm
import numpy as np

In [2]:
def process_data(df, is_drop=True):
    df = df.drop(columns=[
        "Flow ID",
        "Src IP",
    ])

    if is_drop:
        df = df.drop(columns=[
            "Dst IP",
            "Timestamp"
        ])
    else:
        # Timestamp
        import datetime
        # 2017-07-07 11:59:50.315195 -> 1500000000.315195
        DATEFORMAT = "%Y-%m-%d %H:%M:%S.%f"
        df["continuous_timestamp"] = df["Timestamp"].apply(
            lambda x: datetime.strptime(x, DATEFORMAT).timestamp()
        )
        df = df.drop(columns=[
            "Timestamp",
        ])
        df = df.rename(columns={
            "continuous_timestamp": "Timestamp"
        })
        # Dst IP
        import ipaddress as ip
        df["destination_ip"] = df["Dst IP"].apply(
            lambda x: int(ip.IPv4Address(x))
        )
        df = df.drop(columns=[
            "Dst IP",
        ])
        df = df.rename(columns={
            "destination_ip": "Dst IP"
        })
    return df
    

In [3]:
def read_data(pattern=0):
    if pattern == 0:
        file_path = "../data_cicids2017/0_raw/friday.csv"
        df = pd.read_csv(file_path)
        df = process_data(df)
    else:
        directory_path = "../data_cicids2017/0_raw"

        df = pd.DataFrame()
        files_path = glob(f"{directory_path}/*.csv")
        print(files_path)
        for file_path in tqdm(files_path):
            df_tmp = pd.read_csv(file_path)
            df_tmp = process_data(df_tmp)
            df_tmp = df_tmp.replace([np.inf, -np.inf], np.nan)
            df_tmp = df_tmp.dropna()
            df = pd.concat([df, df_tmp], axis=0)
    return df

In [4]:
df = read_data()
X = df.drop(columns=["Label", "Attempted Category"])
y = df["Label"]

print("Before: ", Counter(y))
df = df.drop(columns=[
    'id', 'Src Port',
    'Bwd PSH Flags',
    'Fwd URG Flags', 'Bwd URG Flags', 'Fwd RST Flags', 'Bwd RST Flags',
    'Packet Length Min', 'Packet Length Max',
    'FIN Flag Count',
    'RST Flag Count', 'PSH Flag Count',
    'URG Flag Count', 'CWR Flag Count', 'ECE Flag Count',
    'Fwd Bytes/Bulk Avg', 'Fwd Packet/Bulk Avg', 'Fwd Bulk Rate Avg', 'Bwd Bytes/Bulk Avg',
    'ICMP Code', 'ICMP Type',
    'Total TCP Flow Time',
    'Attempted Category'
])

df["Label"] = df["Label"].apply(lambda x: 0 if x == "BENIGN" else 1)

print("After: ", Counter(df["Label"]))

Before:  Counter({'BENIGN': 288544, 'Portscan': 159066, 'DDoS': 95144, 'Botnet - Attempted': 4067, 'Botnet': 736})
After:  Counter({0: 288544, 1: 259013})


In [5]:
smote_enn = SMOTEENN(
    random_state=42,
    n_jobs=-1,
    smote=SMOTE(
        k_neighbors=2,
        random_state=42,
    )
)

over_x = df.drop(columns=["Label"])
over_y = df["Label"]
print("oversampling start...")
over_x_res, over_y_res = smote_enn.fit_resample(over_x, over_y)
print("oversampling end...")

df = pd.concat([over_x_res, over_y_res], axis=1)

oversampling start...


oversampling end...


In [6]:
print(df.columns)

Index(['Dst Port', 'Protocol', 'Flow Duration', 'Total Fwd Packet',
       'Total Bwd packets', 'Total Length of Fwd Packet',
       'Total Length of Bwd Packet', 'Fwd Packet Length Max',
       'Fwd Packet Length Min', 'Fwd Packet Length Mean',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Mean',
       'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s',
       'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min',
       'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max',
       'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std',
       'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance',
       'SYN Flag Count', 'ACK Flag Count', 'Down/Up Ratio',
       'Average Packet Size', 'Fwd Segment Size Avg', 'Bwd Segment Size Avg',
       'Bw

In [7]:
length = len(df)

ROW_COUNTER = 500_000

i = 0
counter = 0
while i < length:
    counter += 1
    if i + ROW_COUNTER > length:
        df_temp = df.iloc[i:length]
        print(f"{i:10d} - {length:10d}")
    else:
        df_temp = df.iloc[i:i + ROW_COUNTER]
        print(f"{i:10d} - {i + ROW_COUNTER:10d}")

    df_temp.to_csv(f"../data_cicids2017/1_sampling/bi_{counter:03d}_cicids2017.csv", index=False)
    i += ROW_COUNTER

         0 -     500000
    500000 -     576529
